In [113]:
# Домашнее задание

## Задание:
Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA тест, проверьте данные на нормальность и на наличие корреляции. Для этого объедините 2 массива в DataFrame. Обоснуйте выбор теста на корреляцию. Сделайте вывод по гипотезе.
Также ответьте на вопрос:
Какова разница в среднем размере мидии в зависимости от города-производителя. Обоснуйте выбор теста. Сделайте вывод по гипотезе.


In [114]:
# данные:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]

Cписок petersburg имеет 7 значений, в то время как magadan - 8. 
Для объединения в DataFrame списки должны быть одной длины, поэтому добавим в список petersburg
0 в качестве восьмого значения

In [123]:
petersburg.append(0)
print('petersburg', petersburg)
print('длина списка:', len(petersburg), 'элементов')

petersburg [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105, 0, 0]
длина списка: 9 элементов


In [116]:
import pandas as pd # импорт библиотеки для работы с таблицами
# объединим списки в словарь
d = {'petersburg':[0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105, 0], 
     'magadan':[0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]}
data = pd.DataFrame(data=d)
data

,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,0.0000,0.0689


Заменим 0 в признаке petersburg медианным значением

In [117]:
data['petersburg'] = data['petersburg'].apply(lambda x: data['petersburg'].median() if x == 0 else x)
data

,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,0.0995,0.0689


## Тест на нормальность распределения

Применим критерий Шапиро-Уилка, так как он считается более подходящим для небольших выборок.

In [118]:
# cформулируем гипотезу:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (отвергаем H0)'

# Установим уровень значимости:
alpha = 0.05

from scipy.stats import shapiro # импорт нужного модуля

_, p = shapiro(data['petersburg']) # проверка признака petersburg
print('p=%.3f' % p) # вывод значения р

if p > alpha: # сравним р и alpha
	print(H0)
else:
	print(Ha)

p=0.105
Данные распределены нормально


In [119]:
_, p = shapiro(data['magadan']) # проверка признака magadan
print('p=%.3f' % p) # вывод значения р

if p > alpha: # сравним р и alpha
	print(H0)
else:
	print(Ha)

p=0.036
Данные не распределены нормально (отвергаем H0)


### Вывод: в признаке magadan данные имеют ненормальное распределение, поэтому будем использовать непараметрические тесты.

## Корреляционный тест Спирмена

Так как зависимая переменная (средний размер мидий) у нас количественная, используем корреляцию Спирмена.

In [120]:
from scipy.stats import spearmanr # импортируем функцию spearmanr из модуля scipy.stats

corr, p = spearmanr(data['petersburg'], data['magadan'])
# в аргумент функции передадим наши ризнаки
print(f'Коэффициент корреляции: {round(corr, 2)} ; значение Pvalues: {round(p, 2)})')
# выведем значение коэффициента корреляции и Pvalues

if p > 0.05: # сравним Pvalue и alpha
    print('Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными')
else:
    print('Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными.')

Коэффициент корреляции: 0.14 ; значение Pvalues: 0.74)
Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными


### Вывод: тест на корреляцию Спирмена показал, что зависимости в среднем размере мидий, выращенных в разных городах, не существует. Нулевую гипотезу о том, что такой связи не существует, мы не можем отвергнуть.

## Тест на сравнение выборок

Мы сравниваем один и тот же критерий (размер мидий) из двух разных совокупностей (Санкт-Петербург и Магадан), при этом у нас всего 8 значений. Но один из признаков имеет ненормальное распределение, поэтому для сравнения выборок используем Сумму рангов Уилкоксона.

In [121]:
# сформулируем гипотезу
H0 = 'Нет значимой разницы в среднем размере мидии в зависимости от города-производителя.'
Ha = 'Есть значимая разница в среднем размере мидии в зависимости от города-производителя.'

alpha = 0.05 # установим уровень значимости

from scipy.stats import wilcoxon

res = wilcoxon(data['petersburg'], data['magadan'])
# применим функцию wilcoxon(), передав в аргументы признаки для сравнения
print(f'статистики: {res.statistic}, значение Pvalue: {round(res.pvalue, 2)}')
# выведем статистики и значение Pvalue

if res.pvalue > alpha: # сравним Pvalue и alpha
    print(f"{round(res.pvalue, 2)} > {alpha}. {H0}")
else:
    print(f"{round(res.pvalue, 2)} <= {alpha}. {Ha}")

статистики: 2.0, значение Pvalue: 0.02
0.02 <= 0.05. Есть значимая разница в среднем размере мидии в зависимости от города-производителя.


### Вывод: Критерий Уилкоксона показал, что зависимость в среднем размере мидии, выращенных в разных городах, существует. Нулевую гипотезу о том, что такой связи не существует, мы можем отвергнуть.

# Общий вывод:
Для формирования общего вывода о наличии связи среднего размера мидии от города, где они были выращены, следует опираться на данные Суммы рангов Уилкоксона, т.к. этот метод анализирует именно медианные значения выборок. Данные же корреляции Спирмена вполне могут оказаться некорректными, т.к. выборка очень мала.
Соответственно, 
# Разница в среднем размере мидии в зависимости от города-производителя существенна.